In [ ]:
import pandas as pd
import os
import seaborn as sns
import shutil

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_dataframe_terrenos

In [ ]:
estandarizacion_columnas_prediosLimites = ['CODIGO','MATRICULA','Busqueda_VUR_OBSERVACIONES']
estandarizacion_columnas_terrenos = ['codigo','SHAPE']
estandarizacion_prediosLTerrenos = ['codigo','MATRICULA','Busqueda_VUR_OBSERVACIONES','SHAPE']

DIRECTORIO_BD = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\zApoyo_A_Equipos\zApoyoEquipoJuridico\9.Analisis_Predios_LimiteAdministrativo\Base_Datos\02_GeoLimAdmin.gdb"
nombre_capa_prediosLimites = 'prediosLimAdministrativosMariaLaBaja'
RUTA_BD_CAPA_PREDIOS = os.path.join(DIRECTORIO_BD, nombre_capa_prediosLimites)

In [ ]:
sheet_name = 'Predio_LimiteAdministrativo_MariaLaBaja'
sheet_id = '142IZnazQhFzVKgMGvozYSMnFGqoU_QwBVx1yWavGZqA'


url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df_prediosLimites = pd.read_csv(url)
df_prediosLimites = df_prediosLimites[estandarizacion_columnas_prediosLimites]

In [ ]:
resultados = funcion_dataframe_terrenos.dataframe_terrenos()
df_terrenos = resultados[0]
df_terrenos = df_terrenos[estandarizacion_columnas_terrenos]

In [ ]:
df_terrenos

In [ ]:
df_predioLimites_terrenos = pd.merge(left=df_prediosLimites
                                     ,right=df_terrenos
                                     ,left_on='CODIGO'
                                     ,right_on='codigo'
                                     ,how='inner')

print(f"Registros Sheet: {df_prediosLimites.shape[0]}, Registros Cruzados Con Terrenos: {df_predioLimites_terrenos.shape[0]}")

In [ ]:
df_predioLimites_terrenos = df_predioLimites_terrenos[estandarizacion_prediosLTerrenos]

df_predioLimites_terrenos.spatial.to_featureclass(location=RUTA_BD_CAPA_PREDIOS)
print(f"Capa {nombre_capa_prediosLimites} Exportada")

In [ ]:
RUTA_CAPA_TERRENOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\MUNICIPIOS_LOTE4"

arcpy.env.workspace = DIRECTORIO_BD

# ? Disolución Capa PrediosLimiteAdministrativo
nomSalidaCapaDisuelta = 'D_prediosLimAdministrativosMariaLaBaja'
arcpy.Dissolve_management(in_features=RUTA_BD_CAPA_PREDIOS, out_feature_class=nomSalidaCapaDisuelta)
print(f"Capa {nomSalidaCapaDisuelta} Disuelta")

In [ ]:
nomLineaMunicipio = 'L_LimiteAdministrativo'
arcpy.PolygonToLine_management(in_features=RUTA_CAPA_TERRENOS, out_feature_class=nomLineaMunicipio)
print(f"Capa {nomLineaMunicipio} transformada en Línea")

In [ ]:
nomLimiteCorte = 'Clip_LimiteAdministrativo'
arcpy.Clip_analysis(in_features=nomLineaMunicipio, clip_features=nomSalidaCapaDisuelta, out_feature_class=nomLimiteCorte)
print(f"Capa {nomLimiteCorte} Cortada según Predios")

In [ ]:
in_table = nomLimiteCorte
field_name = 'lon_perimetro'
field_alias = 'Longitud Perímetro Mpal Analizado'
field_type = 'DOUBLE'

geometry_property = [[field_name,'LENGTH']]
length_unit = 'KILOMETERS'

arcpy.AddField_management(in_table=in_table,field_name=field_name, field_alias=field_alias,field_type='DOUBLE')
arcpy.CalculateGeometryAttributes_management(in_features=nomLimiteCorte, geometry_property=geometry_property, length_unit=length_unit)